# Profiling `malca.events` on SkyPatrol light curves

Use `cProfile` to time the Bayesian event scorer on real SkyPatrol CSVs stored in `../../input/skypatrol2`. The notebook keeps everything self contained so it can run directly against the repo without extra setup.

## Environment setup

Locate the repo root, add it to `sys.path`, and collect the SkyPatrol CSVs we'll profile.

In [ ]:
import sys
from pathlib import Path

import numpy as np
import pandas as pd

# Find repo root whether the notebook is run from notebooks/ or repo root
candidates = [Path.cwd(), Path.cwd().parent, Path.cwd().parent.parent]
repo_root = next((p for p in candidates if (p / "malca").exists()), Path.cwd())

for path in (repo_root, repo_root / "malca"):
    sp = str(path.resolve())
    if sp not in sys.path:
        sys.path.insert(0, sp)

data_dir = repo_root / "input" / "skypatrol2"
lc_paths = sorted(data_dir.glob("*-light-curves.csv"))

print(f"Repo root: {repo_root}")
print(f"Found {len(lc_paths)} light curves in {data_dir}")
if not lc_paths:
    raise FileNotFoundError("No SkyPatrol CSVs found; adjust data_dir above.")

## Quick peek at one SkyPatrol light curve

Read a single CSV to confirm the loader works and to see the columns that flow into the scorer.

In [ ]:
from malca.plot import read_skypatrol_csv

example_path = lc_paths[0]
df_example = read_skypatrol_csv(example_path)
print(f"{example_path.name}: {len(df_example)} rows")
display(df_example.head())

## Profiling helpers

Wrap `process_one` so we can reuse the same kwargs as the CLI and capture both `cProfile` output and a tidy summary table of the hottest functions.

In [ ]:
import cProfile
import io
import pstats
import time

import malca.events as events

DEFAULT_EVENT_KWARGS = {
    "trigger_mode": "posterior_prob",
    "logbf_threshold_dip": 5.0,
    "logbf_threshold_jump": 5.0,
    "significance_threshold": 99.99997,
    "p_points": 80,
    "p_min_dip": None,
    "p_max_dip": None,
    "p_min_jump": None,
    "p_max_jump": None,
    "mag_points": 12,
    "run_min_points": 3,
    "run_allow_gap_points": 1,
    "run_max_gap_days": None,
    "run_min_duration_days": None,
    "baseline_tag": "gp",
    "use_sigma_eff": True,
    "require_sigma_eff": True,
    "compute_event_prob": True,
}

def score_single_lc(path: Path, **overrides):
    kwargs = dict(DEFAULT_EVENT_KWARGS)
    kwargs.update(overrides)
    return events.process_one(str(path), **kwargs)

def stats_to_frame(stats_obj, limit=20):
    rows = []
    for func, (cc, nc, tt, ct, callers) in stats_obj.stats.items():
        rows.append(
            {
                "func": f"{func[2]} ({Path(func[0]).name}:{func[1]})",
                "ncalls": nc,
                "tottime_s": tt,
                "cumtime_s": ct,
            }
        )
    df_stats = pd.DataFrame(rows)
    if df_stats.empty:
        return df_stats
    return df_stats.sort_values("cumtime_s", ascending=False).head(limit)

def profile_light_curve(path: Path, stats_limit=25, **overrides):
    profiler = cProfile.Profile()
    start = time.perf_counter()
    result = profiler.runcall(score_single_lc, path, **overrides)
    elapsed = time.perf_counter() - start

    stats_buffer = io.StringIO()
    stats_obj = pstats.Stats(profiler, stream=stats_buffer).strip_dirs().sort_stats("cumtime")
    stats_obj.print_stats(stats_limit)
    stats_text = stats_buffer.getvalue()

    top_df = stats_to_frame(stats_obj, limit=stats_limit)
    return result, stats_text, top_df, elapsed

## Profile a single light curve

Run the Bayesian scorer on one SkyPatrol CSV (default parameters match the CLI). `stats_df_single` highlights the functions consuming the most cumulative time.

In [ ]:
single_path = lc_paths[0]
result_single, stats_text_single, stats_df_single, elapsed_single = profile_light_curve(single_path, stats_limit=25)

print(f"Profiled {single_path.name} in {elapsed_single:.2f} s")
print(stats_text_single)
stats_df_single

## Batch timing on a handful of files (optional)

Process a small subset sequentially to gauge throughput without full profiling. Adjust `N_LC` to cover more files if needed.

In [ ]:
N_LC = 3
subset_paths = lc_paths[:N_LC]

timing_rows = []
for path in subset_paths:
    t0 = time.perf_counter()
    res = score_single_lc(path)
    timing_rows.append(
        {
            "path": path.name,
            "elapsed_s": time.perf_counter() - t0,
            "n_points": res.get("n_points"),
            "dip_sig": res.get("dip_significant"),
            "jump_sig": res.get("jump_significant"),
            "dip_bf": res.get("dip_bayes_factor"),
            "jump_bf": res.get("jump_bayes_factor"),
        }
    )

pd.DataFrame(timing_rows)

## Grid resolution sweep
Compare how `p_points` (probability grid) and `mag_points` (magnitude grid) affect runtime and outputs on a small SkyPatrol subset.

In [ ]:
import time
from malca.baseline import per_camera_gp_baseline
import malca.events as events

BASELINE_KWARGS = dict(events.DEFAULT_BASELINE_KWARGS)

def prepare_light_curve(path):
    df_raw = read_skypatrol_csv(path)
    valid_mask = (
        df_raw["JD"].pipe(np.isfinite)
        & df_raw["mag"].pipe(np.isfinite)
        & df_raw["error"].pipe(np.isfinite)
        & (df_raw["error"] > 0)
        & (df_raw["error"] < 10)
    )
    df = df_raw[valid_mask].copy()
    if len(df) < 10:
        raise ValueError(f"Insufficient valid data points ({len(df)}) in {path.name}")
    df = events.clean_lc(df)

    df_base = per_camera_gp_baseline(df, **BASELINE_KWARGS)
    baseline_mags = df_base["baseline"].to_numpy(float) if "baseline" in df_base.columns else df_base["mag"].to_numpy(float)
    baseline_mag = float(np.nanmedian(baseline_mags))
    mags_for_grid = df_base["mag"].to_numpy(float) if "mag" in df_base.columns else df["mag"].to_numpy(float)

    def baseline_precomputed(df_in, **kwargs):
        return df_base

    return df, baseline_precomputed, baseline_mag, mags_for_grid

def run_grid_setting(path, p_points, mag_points, label):
    df, baseline_fn, baseline_mag, mags_for_grid = prepare_light_curve(path)
    mag_grid_dip = events.default_mag_grid(baseline_mag, mags_for_grid, "dip", n=mag_points)
    mag_grid_jump = events.default_mag_grid(baseline_mag, mags_for_grid, "jump", n=mag_points)

    start = time.perf_counter()
    res = events.run_bayesian_significance(
        df,
        baseline_func=baseline_fn,
        baseline_kwargs={},
        p_points=p_points,
        mag_grid_dip=mag_grid_dip,
        mag_grid_jump=mag_grid_jump,
        trigger_mode="posterior_prob",
        logbf_threshold_dip=5.0,
        logbf_threshold_jump=5.0,
        significance_threshold=99.99997,
        run_min_points=3,
        run_allow_gap_points=1,
        run_max_gap_days=None,
        run_min_duration_days=None,
        use_sigma_eff=True,
        require_sigma_eff=True,
        compute_event_prob=True,
    )
    elapsed = time.perf_counter() - start

    return {
        "path": path.name,
        "config": label,
        "p_points": p_points,
        "mag_points": mag_points,
        "elapsed_s": elapsed,
        "dip_sig": res["dip"]["significant"],
        "jump_sig": res["jump"]["significant"],
        "dip_bf": res["dip"]["bayes_factor"],
        "jump_bf": res["jump"]["bayes_factor"],
        "dip_best_p": res["dip"]["best_p"],
        "jump_best_p": res["jump"]["best_p"],
    }

In [ ]:
# Evaluate a few grid settings on a small subset of light curves
N_LC = 3
subset_paths = lc_paths[:N_LC]

grid_settings = [
    {"label": "baseline_80x60", "p_points": 80, "mag_points": 60},
    {"label": "coarse_40x30", "p_points": 40, "mag_points": 30},
    {"label": "fine_p_160x60", "p_points": 160, "mag_points": 60},
    {"label": "fine_mag_80x120", "p_points": 80, "mag_points": 120},
]

rows = []
for path in subset_paths:
    for cfg in grid_settings:
        rows.append(run_grid_setting(path, cfg["p_points"], cfg["mag_points"], cfg["label"]))

df_grid = pd.DataFrame(rows)
df_grid

In [ ]:
# Compare each setting to the baseline configuration
baseline_label = "baseline_80x60"
base = df_grid[df_grid["config"] == baseline_label]
comparison = df_grid.merge(base, on="path", suffixes=("", "_base"))
comparison["dip_bf_delta"] = comparison["dip_bf"] - comparison["dip_bf_base"]
comparison["jump_bf_delta"] = comparison["jump_bf"] - comparison["jump_bf_base"]
comparison["elapsed_delta_s"] = comparison["elapsed_s"] - comparison["elapsed_s_base"]
cols = [
    "path",
    "config",
    "p_points",
    "mag_points",
    "elapsed_s",
    "elapsed_delta_s",
    "dip_sig",
    "dip_sig_base",
    "dip_bf",
    "dip_bf_delta",
    "jump_sig",
    "jump_sig_base",
    "jump_bf",
    "jump_bf_delta",
]
comparison[cols]

## Extended coarse grid sweep
Test a wider range of grid configurations, especially at the coarse end, to find the optimal speed/accuracy tradeoff.

In [ ]:
# Build simulated light curves by injecting analytic events into a real cadence
from malca import events

rng = np.random.default_rng(42)

base_sim_path = lc_paths[0]
df_sim_base_raw = read_skypatrol_csv(base_sim_path)
mask = (
    df_sim_base_raw["JD"].pipe(np.isfinite)
    & df_sim_base_raw["mag"].pipe(np.isfinite)
    & df_sim_base_raw["error"].pipe(np.isfinite)
    & (df_sim_base_raw["error"] > 0)
    & (df_sim_base_raw["error"] < 10)
)
df_sim_base = df_sim_base_raw[mask].copy()
df_sim_base = events.clean_lc(df_sim_base)

jd_med = float(df_sim_base["JD"].median())
jd_span = float(df_sim_base["JD"].max() - df_sim_base["JD"].min())


def inject_event(df_in, kind="dip", shape="gaussian", amp=0.25, width=25.0, t0_offset=0.0):
    # Return a copy with an injected event (positive amp = dip, negative amp = jump).
    df = df_in.copy()
    t0 = jd_med + float(t0_offset)
    amp_signed = float(amp) if kind == "dip" else -float(amp)
    t_arr = df["JD"].to_numpy(float)
    if shape == "gaussian":
        delta = events.gaussian(t_arr, amp_signed, t0, float(width), 0.0)
    elif shape == "paczynski":
        delta = events.paczynski(t_arr, amp_signed, t0, float(width), 0.0)
    else:
        raise ValueError("shape must be gaussian or paczynski")
    df["mag"] = df["mag"].to_numpy(float) + delta
    return df


def inject_mixed(df_in):
    df = inject_event(df_in, kind="dip", shape="gaussian", amp=0.18, width=20.0, t0_offset=-60.0)
    df = inject_event(df, kind="jump", shape="paczynski", amp=0.14, width=30.0, t0_offset=40.0)
    return df


def add_noise_and_gaps(df_in, jitter_mag=0.02, gap_frac=0.15, spike_amp=0.25, spike_width=0.08, spike_count=3):
    # Add photometric jitter, drop random points, and sprinkle outliers to make messy cases.
    df = df_in.copy()
    n = len(df)
    df["mag"] = df["mag"].to_numpy(float) + rng.normal(0.0, jitter_mag, n)
    df["error"] = (df["error"].to_numpy(float) * (1 + rng.normal(0.0, 0.05, n))).clip(min=1e-3)
    keep = rng.random(n) > gap_frac
    df = df.loc[keep].copy().reset_index(drop=True)
    if len(df) == 0:
        return df_in.copy()
    for _ in range(int(spike_count)):
        idx = int(rng.integers(0, len(df)))
        df.loc[idx, "mag"] += rng.normal(spike_amp, spike_width)
    df = df.sort_values("JD").reset_index(drop=True)
    return df

# MASSIVELY EXPANDED SIMULATED LIGHT CURVES
simulated_lcs = {}

# ========== SINGLE DIPS - GAUSSIAN ==========
# Amplitude sweep (shallow to deep)
for amp in [0.05, 0.08, 0.10, 0.12, 0.15, 0.18, 0.20, 0.25, 0.30, 0.40, 0.50]:
    simulated_lcs[f"dip_gauss_a{amp:.2f}_w20"] = inject_event(df_sim_base, "dip", "gaussian", amp=amp, width=20.0, t0_offset=0.0)

# Width sweep (fast to slow)
for width in [3.0, 5.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 70.0, 100.0]:
    simulated_lcs[f"dip_gauss_a0.20_w{width:.0f}"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.20, width=width, t0_offset=0.0)

# Position sweep (early, middle, late)
for offset in [-jd_span*0.4, -jd_span*0.2, 0.0, jd_span*0.2, jd_span*0.4]:
    simulated_lcs[f"dip_gauss_a0.20_w20_t{offset:.0f}"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.20, width=20.0, t0_offset=offset)

# ========== SINGLE DIPS - PACZYNSKI ==========
# Amplitude sweep
for amp in [0.05, 0.08, 0.10, 0.12, 0.15, 0.18, 0.20, 0.25, 0.30, 0.40, 0.50]:
    simulated_lcs[f"dip_pacz_a{amp:.2f}_w20"] = inject_event(df_sim_base, "dip", "paczynski", amp=amp, width=20.0, t0_offset=0.0)

# Width sweep
for width in [3.0, 5.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 70.0]:
    simulated_lcs[f"dip_pacz_a0.20_w{width:.0f}"] = inject_event(df_sim_base, "dip", "paczynski", amp=0.20, width=width, t0_offset=0.0)

# ========== SINGLE JUMPS - GAUSSIAN ==========
# Amplitude sweep
for amp in [0.05, 0.08, 0.10, 0.12, 0.15, 0.18, 0.20, 0.25, 0.30, 0.40]:
    simulated_lcs[f"jump_gauss_a{amp:.2f}_w20"] = inject_event(df_sim_base, "jump", "gaussian", amp=amp, width=20.0, t0_offset=0.0)

# Width sweep
for width in [5.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 60.0, 80.0]:
    simulated_lcs[f"jump_gauss_a0.20_w{width:.0f}"] = inject_event(df_sim_base, "jump", "gaussian", amp=0.20, width=width, t0_offset=0.0)

# ========== SINGLE JUMPS - PACZYNSKI (MICROLENSING) ==========
# Amplitude sweep
for amp in [0.05, 0.08, 0.10, 0.12, 0.15, 0.18, 0.20, 0.25, 0.30, 0.40, 0.50]:
    simulated_lcs[f"jump_pacz_a{amp:.2f}_w20"] = inject_event(df_sim_base, "jump", "paczynski", amp=amp, width=20.0, t0_offset=0.0)

# Width sweep (microlensing timescales)
for width in [5.0, 8.0, 10.0, 15.0, 20.0, 25.0, 30.0, 40.0, 50.0, 70.0, 100.0]:
    simulated_lcs[f"jump_pacz_a0.20_w{width:.0f}"] = inject_event(df_sim_base, "jump", "paczynski", amp=0.20, width=width, t0_offset=0.0)

# ========== DOUBLE EVENTS ==========
# Two dips - varying separation
for sep in [50.0, 80.0, 120.0, 180.0]:
    df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=0.18, width=15.0, t0_offset=-sep/2)
    simulated_lcs[f"dip_double_sep{sep:.0f}"] = inject_event(df_temp, "dip", "gaussian", amp=0.15, width=12.0, t0_offset=sep/2)

# Two jumps
for sep in [60.0, 100.0, 150.0]:
    df_temp = inject_event(df_sim_base, "jump", "paczynski", amp=0.15, width=18.0, t0_offset=-sep/2)
    simulated_lcs[f"jump_double_sep{sep:.0f}"] = inject_event(df_temp, "jump", "paczynski", amp=0.13, width=15.0, t0_offset=sep/2)

# Dip then jump
for sep in [60.0, 100.0, 140.0]:
    df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=0.18, width=20.0, t0_offset=-sep/2)
    simulated_lcs[f"dip_then_jump_sep{sep:.0f}"] = inject_event(df_temp, "jump", "paczynski", amp=0.14, width=25.0, t0_offset=sep/2)

# Jump then dip
for sep in [60.0, 100.0]:
    df_temp = inject_event(df_sim_base, "jump", "gaussian", amp=0.16, width=22.0, t0_offset=-sep/2)
    simulated_lcs[f"jump_then_dip_sep{sep:.0f}"] = inject_event(df_temp, "dip", "paczynski", amp=0.15, width=18.0, t0_offset=sep/2)

# ========== TRIPLE EVENTS ==========
# Three dips
df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=0.12, width=12.0, t0_offset=-100.0)
df_temp = inject_event(df_temp, "dip", "gaussian", amp=0.15, width=15.0, t0_offset=0.0)
simulated_lcs["dip_triple"] = inject_event(df_temp, "dip", "gaussian", amp=0.10, width=10.0, t0_offset=100.0)

# ========== NOISY VERSIONS ==========
# Low noise
for case in ["dip_gauss_a0.20_w20", "dip_pacz_a0.20_w20", "jump_gauss_a0.20_w20", "jump_pacz_a0.20_w20"]:
    if case in simulated_lcs:
        simulated_lcs[f"{case}_noise_low"] = add_noise_and_gaps(simulated_lcs[case], jitter_mag=0.01, gap_frac=0.05, spike_count=1)

# Medium noise
for case in ["dip_gauss_a0.20_w20", "dip_pacz_a0.20_w20", "jump_gauss_a0.20_w20", "jump_pacz_a0.20_w20"]:
    if case in simulated_lcs:
        simulated_lcs[f"{case}_noise_med"] = add_noise_and_gaps(simulated_lcs[case], jitter_mag=0.02, gap_frac=0.12, spike_count=2)

# High noise
for case in ["dip_gauss_a0.20_w20", "dip_pacz_a0.20_w20", "jump_gauss_a0.20_w20", "jump_pacz_a0.20_w20"]:
    if case in simulated_lcs:
        simulated_lcs[f"{case}_noise_high"] = add_noise_and_gaps(simulated_lcs[case], jitter_mag=0.04, gap_frac=0.25, spike_count=5)

# Very high noise (challenging)
for case in ["dip_gauss_a0.20_w20", "dip_pacz_a0.20_w20", "jump_gauss_a0.20_w20", "jump_pacz_a0.20_w20"]:
    if case in simulated_lcs:
        simulated_lcs[f"{case}_noise_extreme"] = add_noise_and_gaps(simulated_lcs[case], jitter_mag=0.06, gap_frac=0.35, spike_count=8)

# ========== SHALLOW + NOISY (DETECTION LIMITS) ==========
for amp in [0.05, 0.08, 0.10]:
    for noise in [0.01, 0.02, 0.03]:
        df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=amp, width=20.0, t0_offset=0.0)
        simulated_lcs[f"dip_shallow_a{amp:.2f}_n{noise:.2f}"] = add_noise_and_gaps(df_temp, jitter_mag=noise, gap_frac=0.1, spike_count=2)

# ========== FAST TRANSIENTS ==========
for width in [2.0, 3.0, 4.0, 5.0]:
    simulated_lcs[f"dip_fast_w{width:.0f}"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.25, width=width, t0_offset=0.0)
    simulated_lcs[f"jump_fast_w{width:.0f}"] = inject_event(df_sim_base, "jump", "gaussian", amp=0.20, width=width, t0_offset=0.0)

# ========== SLOW TRANSIENTS ==========
for width in [80.0, 100.0, 120.0, 150.0]:
    simulated_lcs[f"dip_slow_w{width:.0f}"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.22, width=width, t0_offset=0.0)
    simulated_lcs[f"jump_slow_w{width:.0f}"] = inject_event(df_sim_base, "jump", "paczynski", amp=0.20, width=width, t0_offset=0.0)

# ========== ASYMMETRIC EVENTS (MIXED SHAPES) ==========
# Gaussian dip + Paczynski jump
df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=0.18, width=18.0, t0_offset=-70.0)
simulated_lcs["asym_gauss_dip_pacz_jump"] = inject_event(df_temp, "jump", "paczynski", amp=0.16, width=25.0, t0_offset=50.0)

# Paczynski dip + Gaussian jump
df_temp = inject_event(df_sim_base, "dip", "paczynski", amp=0.20, width=22.0, t0_offset=-80.0)
simulated_lcs["asym_pacz_dip_gauss_jump"] = inject_event(df_temp, "jump", "gaussian", amp=0.15, width=20.0, t0_offset=60.0)

# ========== EXTREME CASES ==========
# Very deep dip
simulated_lcs["dip_very_deep"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.70, width=25.0, t0_offset=0.0)

# Very shallow dip (near detection limit)
simulated_lcs["dip_very_shallow"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.03, width=20.0, t0_offset=0.0)

# Very fast dip
simulated_lcs["dip_very_fast"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.25, width=1.5, t0_offset=0.0)

# Very slow dip
simulated_lcs["dip_very_slow"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.20, width=200.0, t0_offset=0.0)

# Strong microlensing
simulated_lcs["microlens_strong"] = inject_event(df_sim_base, "jump", "paczynski", amp=0.60, width=30.0, t0_offset=0.0)

# Weak microlensing
simulated_lcs["microlens_weak"] = inject_event(df_sim_base, "jump", "paczynski", amp=0.04, width=15.0, t0_offset=0.0)

# ========== EDGE CASES ==========
# Event at beginning
simulated_lcs["dip_at_start"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.22, width=20.0, t0_offset=-jd_span*0.45)

# Event at end
simulated_lcs["dip_at_end"] = inject_event(df_sim_base, "dip", "gaussian", amp=0.22, width=20.0, t0_offset=jd_span*0.45)

# Multiple small events
df_temp = df_sim_base.copy()
for offset in [-120.0, -60.0, 0.0, 60.0, 120.0]:
    df_temp = inject_event(df_temp, "dip", "gaussian", amp=0.08, width=10.0, t0_offset=offset)
simulated_lcs["dip_many_small"] = df_temp

# Overlapping events
df_temp = inject_event(df_sim_base, "dip", "gaussian", amp=0.20, width=30.0, t0_offset=-10.0)
simulated_lcs["dip_overlapping"] = inject_event(df_temp, "dip", "gaussian", amp=0.15, width=25.0, t0_offset=10.0)

print(f"Created {len(simulated_lcs)} simulated light curves")
print(f"\\nCategories:")
print(f"  Single dips (Gaussian): {len([k for k in simulated_lcs.keys() if k.startswith('dip_gauss') and 'noise' not in k and 'shallow' not in k])}")
print(f"  Single dips (Paczynski): {len([k for k in simulated_lcs.keys() if k.startswith('dip_pacz') and 'noise' not in k])}")
print(f"  Single jumps (Gaussian): {len([k for k in simulated_lcs.keys() if k.startswith('jump_gauss') and 'noise' not in k])}")
print(f"  Single jumps (Paczynski): {len([k for k in simulated_lcs.keys() if k.startswith('jump_pacz') and 'noise' not in k])}")
print(f"  Double/Triple events: {len([k for k in simulated_lcs.keys() if 'double' in k or 'triple' in k])}")
print(f"  Noisy variants: {len([k for k in simulated_lcs.keys() if 'noise' in k])}")
print(f"  Shallow+noisy: {len([k for k in simulated_lcs.keys() if 'shallow' in k])}")
print(f"  Fast/slow transients: {len([k for k in simulated_lcs.keys() if 'fast' in k or 'slow' in k])}")
print(f"  Edge cases: {len([k for k in simulated_lcs.keys() if any(x in k for x in ['extreme', 'asym', 'start', 'end', 'many', 'overlap'])])}")
print(f"\\nSample keys: {list(simulated_lcs.keys())[:10]}")
display(df_sim_base.head())

## Comprehensive grid performance on simulated light curves
Run all extended grid configurations on the full simulated light curve suite and analyze detection accuracy with emphasis on false positives.

In [ ]:
# Determine ground truth for each simulated light curve
def get_ground_truth(lc_name):
    """Return expected detection: ('dip', 'jump', 'both', 'none')"""
    name_lower = lc_name.lower()
    
    # Cases with both dip and jump
    if any(x in name_lower for x in ['mixed', 'dip_then_jump', 'jump_then_dip', 'asym']):
        return 'both'
    
    # Dip-only cases
    elif 'dip' in name_lower:
        # Very shallow dips may be below detection threshold
        if 'very_shallow' in name_lower or ('shallow' in name_lower and 'a0.03' in name_lower):
            return 'dip_marginal'  # May or may not be detected
        return 'dip'
    
    # Jump-only cases (including microlensing)
    elif 'jump' in name_lower or 'microlens' in name_lower:
        # Very weak microlensing may be below threshold
        if 'microlens_weak' in name_lower or ('weak' in name_lower and 'a0.04' in name_lower):
            return 'jump_marginal'
        return 'jump'
    
    # No injected events
    else:
        return 'none'


# Build ground truth dictionary
ground_truth = {name: get_ground_truth(name) for name in simulated_lcs.keys()}

# Preview ground truth distribution
from collections import Counter
truth_counts = Counter(ground_truth.values())
print("Ground truth distribution:")
for category, count in sorted(truth_counts.items()):
    print(f"  {category}: {count}")
    
print(f"\nExamples:")
for name in list(simulated_lcs.keys())[:10]:
    print(f"  {name}: {ground_truth[name]}")

In [ ]:
# Define all 65 extended grid configurations (if not already defined)
if 'extended_grid_configs' not in globals():
    extended_grid_configs = []

    # Absurdly coarse (2x2 to 10x10)
    for n in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
        extended_grid_configs.append({"label": f"absurd_{n}x{n}", "p_points": n, "mag_points": n})

    # Ultra coarse (12x12 to 20x20)
    for n in [12, 15, 18, 20]:
        extended_grid_configs.append({"label": f"ultra_coarse_{n}x{n}", "p_points": n, "mag_points": n})

    # Very coarse (22x22 to 35x35)
    for n in [22, 25, 28, 30, 35]:
        extended_grid_configs.append({"label": f"very_coarse_{n}x{n}", "p_points": n, "mag_points": n})

    # Coarse (asymmetric versions)
    for p, m in [(20, 15), (25, 20), (30, 20), (30, 25), (35, 25), (40, 25), (40, 30), (45, 30), (50, 35)]:
        extended_grid_configs.append({"label": f"coarse_{p}x{m}", "p_points": p, "mag_points": m})

    # Medium (50x40 to 70x60)
    for p, m in [(50, 40), (55, 45), (60, 45), (60, 50), (65, 50), (70, 50), (70, 60)]:
        extended_grid_configs.append({"label": f"medium_{p}x{m}", "p_points": p, "mag_points": m})

    # Baseline and near-baseline
    extended_grid_configs.append({"label": "baseline_80x60", "p_points": 80, "mag_points": 60})
    for p, m in [(75, 55), (85, 65), (90, 65), (90, 70)]:
        extended_grid_configs.append({"label": f"near_base_{p}x{m}", "p_points": p, "mag_points": m})

    # Fine (95x70 to 120x100)
    for p, m in [(95, 70), (100, 75), (100, 80), (105, 80), (110, 85), (110, 90), (115, 90), (120, 90), (120, 100)]:
        extended_grid_configs.append({"label": f"fine_{p}x{m}", "p_points": p, "mag_points": m})

    print(f"Created {len(extended_grid_configs)} grid configurations")
    print(f"Grid sizes range from {min(c['p_points']*c['mag_points'] for c in extended_grid_configs)} to {max(c['p_points']*c['mag_points'] for c in extended_grid_configs)}")
else:
    print(f"Using existing extended_grid_configs with {len(extended_grid_configs)} configurations")

In [ ]:
# Define helper function for running grid configs on simulated light curves
from malca.baseline import per_camera_gp_baseline

def prepare_df_for_sim(df):
    df_clean = events.clean_lc(df)
    df_base = per_camera_gp_baseline(df_clean, **BASELINE_KWARGS)
    baseline_mags = df_base["baseline"].to_numpy(float) if "baseline" in df_base.columns else df_base["mag"].to_numpy(float)
    baseline_mag = float(np.nanmedian(baseline_mags))
    mags_for_grid = df_base["mag"].to_numpy(float) if "mag" in df_base.columns else df_clean["mag"].to_numpy(float)

    def baseline_precomputed(df_in, **kwargs):
        return df_base

    return df_clean, baseline_precomputed, baseline_mag, mags_for_grid


def run_grid_setting_sim(name, df, p_points, mag_points, label):
    df_clean, baseline_fn, baseline_mag, mags_for_grid = prepare_df_for_sim(df)
    mag_grid_dip = events.default_mag_grid(baseline_mag, mags_for_grid, "dip", n=mag_points)
    mag_grid_jump = events.default_mag_grid(baseline_mag, mags_for_grid, "jump", n=mag_points)

    start = time.perf_counter()
    res = events.run_bayesian_significance(
        df_clean,
        baseline_func=baseline_fn,
        baseline_kwargs={},
        p_points=p_points,
        mag_grid_dip=mag_grid_dip,
        mag_grid_jump=mag_grid_jump,
        trigger_mode="posterior_prob",
        logbf_threshold_dip=5.0,
        logbf_threshold_jump=5.0,
        significance_threshold=99.99997,
        run_min_points=3,
        run_allow_gap_points=1,
        run_max_gap_days=None,
        run_min_duration_days=None,
        use_sigma_eff=True,
        require_sigma_eff=True,
        compute_event_prob=True,
    )
    elapsed = time.perf_counter() - start

    return {
        "case": name,
        "config": label,
        "p_points": p_points,
        "mag_points": mag_points,
        "elapsed_s": elapsed,
        "dip_sig": res["dip"]["significant"],
        "jump_sig": res["jump"]["significant"],
        "dip_bf": res["dip"]["bayes_factor"],
        "jump_bf": res["jump"]["bayes_factor"],
        "dip_best_p": res["dip"]["best_p"],
        "jump_best_p": res["jump"]["best_p"],
    }

print("Helper functions defined: prepare_df_for_sim, run_grid_setting_sim")

In [ ]:
# Run all extended grid configs on all simulated light curves (PARALLELIZED)
# Using 10 cores to speed up computation
import time
from joblib import Parallel, delayed
from tqdm.auto import tqdm

# Create all task combinations upfront
tasks = []
for cfg in extended_grid_configs:
    for lc_name, df_sim in simulated_lcs.items():
        tasks.append({
            'cfg': cfg,
            'lc_name': lc_name,
            'df_sim': df_sim,
            'truth': ground_truth[lc_name]
        })

print(f"Running {len(extended_grid_configs)} configs on {len(simulated_lcs)} simulated LCs")
print(f"Total tasks: {len(tasks)}")
print(f"Using 16 parallel workers")
print(f"Estimated time: ~{len(tasks) * 6 * 0.15 / 60 / 15:.1f} minutes with 15 cores")
print()

# Worker function for parallel execution
def process_single_task(task):
    """Process one (config, light_curve) combination"""
    try:
        result = run_grid_setting_sim(
            task['lc_name'], 
            task['df_sim'], 
            task['cfg']["p_points"], 
            task['cfg']["mag_points"], 
            task['cfg']["label"]
        )
        result['truth'] = task['truth']
        return result, None
    except Exception as e:
        return None, {'cfg': task['cfg']['label'], 'lc': task['lc_name'], 'error': str(e)}

# Run in parallel with progress bar
start_time = time.time()

results = Parallel(n_jobs=15, verbose=0)(
    delayed(process_single_task)(task) 
    for task in tqdm(tasks, desc="Processing", unit="task")
)

total_time = time.time() - start_time

# Separate successful results from errors
sim_extended_rows = []
errors = []
for result, error in results:
    if result is not None:
        sim_extended_rows.append(result)
    if error is not None:
        errors.append(error)

# Report errors if any
if len(errors) > 0:
    print(f"\n⚠ {len(errors)} errors occurred:")
    for err in errors[:10]:  # Show first 10
        print(f"  [{err['cfg']}] on {err['lc']}: {err['error']}")
    if len(errors) > 10:
        print(f"  ... and {len(errors) - 10} more errors")

df_sim_extended = pd.DataFrame(sim_extended_rows)

print(f"\n{'='*80}")
print(f"COMPLETED: {len(df_sim_extended)} results in {total_time/60:.1f} minutes")
print(f"Average: {total_time/len(df_sim_extended):.3f}s per run")
print(f"Speedup: ~{len(tasks) * 0.15 / total_time:.1f}x compared to sequential")
print(f"{'='*80}")
df_sim_extended.head()

In [ ]:
# Compute detection metrics for each result
def classify_detection(row):
    """
    Classify detection as TP, FP, FN, TN for both dip and jump
    Returns dict with metrics
    """
    truth = row['truth']
    detected_dip = row['dip_sig']
    detected_jump = row['jump_sig']
    
    metrics = {}
    
    # DIP METRICS
    if truth in ['dip', 'both']:
        # Should detect dip
        metrics['dip_tp'] = 1 if detected_dip else 0
        metrics['dip_fn'] = 0 if detected_dip else 1
        metrics['dip_fp'] = 0
        metrics['dip_tn'] = 0
    elif truth in ['dip_marginal']:
        # Marginal case - don't penalize either way
        metrics['dip_tp'] = 0
        metrics['dip_fn'] = 0
        metrics['dip_fp'] = 0
        metrics['dip_tn'] = 0
    else:
        # Should NOT detect dip
        metrics['dip_tp'] = 0
        metrics['dip_fn'] = 0
        metrics['dip_fp'] = 1 if detected_dip else 0
        metrics['dip_tn'] = 0 if detected_dip else 1
    
    # JUMP METRICS
    if truth in ['jump', 'both']:
        # Should detect jump
        metrics['jump_tp'] = 1 if detected_jump else 0
        metrics['jump_fn'] = 0 if detected_jump else 1
        metrics['jump_fp'] = 0
        metrics['jump_tn'] = 0
    elif truth in ['jump_marginal']:
        # Marginal case - don't penalize either way
        metrics['jump_tp'] = 0
        metrics['jump_fn'] = 0
        metrics['jump_fp'] = 0
        metrics['jump_tn'] = 0
    else:
        # Should NOT detect jump
        metrics['jump_tp'] = 0
        metrics['jump_fn'] = 0
        metrics['jump_fp'] = 1 if detected_jump else 0
        metrics['jump_tn'] = 0 if detected_jump else 1
    
    # COMBINED: Any false positive
    metrics['any_fp'] = 1 if (metrics['dip_fp'] > 0 or metrics['jump_fp'] > 0) else 0
    
    return metrics

# Apply classification to all results
metrics_rows = []
for idx, row in df_sim_extended.iterrows():
    metrics = classify_detection(row)
    metrics['config'] = row['config']
    metrics['case'] = row['case']
    metrics['truth'] = row['truth']
    metrics['elapsed_s'] = row['elapsed_s']
    metrics['dip_bf'] = row['dip_bf']
    metrics['jump_bf'] = row['jump_bf']
    metrics_rows.append(metrics)

df_metrics = pd.DataFrame(metrics_rows)

print(f"Computed metrics for {len(df_metrics)} results")
df_metrics.head(10)

In [ ]:
# Aggregate metrics by configuration
config_metrics = df_metrics.groupby('config').agg({
    'dip_tp': 'sum',
    'dip_fp': 'sum',
    'dip_fn': 'sum',
    'dip_tn': 'sum',
    'jump_tp': 'sum',
    'jump_fp': 'sum',
    'jump_fn': 'sum',
    'jump_tn': 'sum',
    'any_fp': 'sum',
    'elapsed_s': 'mean',
}).reset_index()

# Calculate derived metrics
config_metrics['dip_precision'] = config_metrics['dip_tp'] / (config_metrics['dip_tp'] + config_metrics['dip_fp']).replace(0, np.nan)
config_metrics['dip_recall'] = config_metrics['dip_tp'] / (config_metrics['dip_tp'] + config_metrics['dip_fn']).replace(0, np.nan)
config_metrics['dip_f1'] = 2 * (config_metrics['dip_precision'] * config_metrics['dip_recall']) / (config_metrics['dip_precision'] + config_metrics['dip_recall'])
config_metrics['dip_fpr'] = config_metrics['dip_fp'] / (config_metrics['dip_fp'] + config_metrics['dip_tn']).replace(0, np.nan)

config_metrics['jump_precision'] = config_metrics['jump_tp'] / (config_metrics['jump_tp'] + config_metrics['jump_fp']).replace(0, np.nan)
config_metrics['jump_recall'] = config_metrics['jump_tp'] / (config_metrics['jump_tp'] + config_metrics['jump_fn']).replace(0, np.nan)
config_metrics['jump_f1'] = 2 * (config_metrics['jump_precision'] * config_metrics['jump_recall']) / (config_metrics['jump_precision'] + config_metrics['jump_recall'])
config_metrics['jump_fpr'] = config_metrics['jump_fp'] / (config_metrics['jump_fp'] + config_metrics['jump_tn']).replace(0, np.nan)

# Overall metrics
config_metrics['total_tp'] = config_metrics['dip_tp'] + config_metrics['jump_tp']
config_metrics['total_fp'] = config_metrics['dip_fp'] + config_metrics['jump_fp']
config_metrics['total_fn'] = config_metrics['dip_fn'] + config_metrics['jump_fn']
config_metrics['overall_precision'] = config_metrics['total_tp'] / (config_metrics['total_tp'] + config_metrics['total_fp']).replace(0, np.nan)
config_metrics['overall_recall'] = config_metrics['total_tp'] / (config_metrics['total_tp'] + config_metrics['total_fn']).replace(0, np.nan)
config_metrics['overall_f1'] = 2 * (config_metrics['overall_precision'] * config_metrics['overall_recall']) / (config_metrics['overall_precision'] + config_metrics['overall_recall'])

# Get grid size info from extended_grid_configs
config_info = {cfg['label']: {'p_points': cfg['p_points'], 'mag_points': cfg['mag_points']} for cfg in extended_grid_configs}
config_metrics['p_points'] = config_metrics['config'].map(lambda x: config_info.get(x, {}).get('p_points', np.nan))
config_metrics['mag_points'] = config_metrics['config'].map(lambda x: config_info.get(x, {}).get('mag_points', np.nan))
config_metrics['grid_size'] = config_metrics['p_points'] * config_metrics['mag_points']

# Sort by false positive rate (ascending - lower is better)
config_metrics_sorted = config_metrics.sort_values('total_fp')

print("="*80)
print("CONFIGURATION PERFORMANCE SUMMARY")
print("="*80)
print(f"\nTotal configurations tested: {len(config_metrics)}")
print(f"Total test cases per config: {len(simulated_lcs)}")
print(f"\nTop 10 configs by LOWEST false positives:")
print(config_metrics_sorted[['config', 'grid_size', 'total_fp', 'dip_fp', 'jump_fp', 
                              'overall_precision', 'overall_recall', 'overall_f1', 'elapsed_s']].head(10))

print(f"\n\nTop 10 configs by HIGHEST false positives (worst performers):")
print(config_metrics.sort_values('total_fp', ascending=False)[['config', 'grid_size', 'total_fp', 'dip_fp', 'jump_fp', 
                                                                 'overall_precision', 'overall_recall', 'overall_f1', 'elapsed_s']].head(10))

config_metrics_sorted.head()

In [ ]:
# Detailed false positive analysis
print("="*80)
print("FALSE POSITIVE DEEP DIVE")
print("="*80)

# Which light curves trigger the most false positives across all configs?
fp_by_case = df_metrics[df_metrics['any_fp'] > 0].groupby('case').agg({
    'any_fp': 'sum',  # How many configs falsely detected this case
    'truth': 'first',
}).sort_values('any_fp', ascending=False)

print(f"\nLight curves with most false positive detections (across all {len(config_metrics)} configs):")
print(fp_by_case.head(20))

# Which types of events are most prone to false positives?
fp_by_truth = df_metrics[df_metrics['any_fp'] > 0].groupby('truth').agg({
    'any_fp': 'sum',
    'dip_fp': 'sum',
    'jump_fp': 'sum',
}).sort_values('any_fp', ascending=False)

print(f"\n\nFalse positives by ground truth category:")
print(fp_by_truth)

# For baseline config specifically
baseline_fps = df_metrics[df_metrics['config'] == 'baseline_80x60']
baseline_fp_cases = baseline_fps[baseline_fps['any_fp'] > 0].sort_values('case')

print(f"\n\nBaseline (80x60) false positives: {baseline_fps['any_fp'].sum()} total")
print(f"Breakdown:")
print(f"  Dip false positives: {baseline_fps['dip_fp'].sum()}")
print(f"  Jump false positives: {baseline_fps['jump_fp'].sum()}")

if len(baseline_fp_cases) > 0:
    print(f"\nBaseline false positive cases ({len(baseline_fp_cases)} cases):")
    print(baseline_fp_cases[['case', 'truth', 'dip_fp', 'jump_fp', 'dip_bf', 'jump_bf']].head(15))
else:
    print("\n✓ Baseline has ZERO false positives - excellent!")

# Compare false positive rates across grid size ranges
config_metrics['size_category'] = pd.cut(config_metrics['grid_size'], 
                                          bins=[0, 100, 500, 1500, 3000, 15000],
                                          labels=['tiny (<100)', 'small (100-500)', 'medium (500-1500)', 
                                                  'large (1500-3000)', 'very large (>3000)'])

fp_by_size = config_metrics.groupby('size_category').agg({
    'total_fp': 'mean',
    'dip_fp': 'mean',
    'jump_fp': 'mean',
    'overall_precision': 'mean',
    'overall_recall': 'mean',
    'elapsed_s': 'mean',
    'config': 'count',
})

print(f"\n\nFalse positive rates by grid size category:")
print(fp_by_size)

In [ ]:
# Comprehensive visualizations emphasizing false positives
import matplotlib.pyplot as plt

fig, axes = plt.subplots(3, 2, figsize=(18, 20))

# Plot 1: Grid size vs False Positives
ax = axes[0, 0]
scatter = ax.scatter(config_metrics['grid_size'], config_metrics['total_fp'], 
                     s=120, alpha=0.6, c=config_metrics['elapsed_s'], cmap='viridis')
ax.set_xlabel('Grid Size (p_points × mag_points)', fontsize=12)
ax.set_ylabel('Total False Positives', fontsize=12)
ax.set_title('Grid Size vs False Positive Count', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
plt.colorbar(scatter, ax=ax, label='Avg Time (s)')
# Annotate baseline
baseline_row = config_metrics[config_metrics['config'] == 'baseline_80x60'].iloc[0]
ax.annotate('baseline\n80×60', (baseline_row['grid_size'], baseline_row['total_fp']),
            fontsize=10, fontweight='bold', color='red',
            bbox=dict(boxstyle='round', facecolor='white', alpha=0.7))

# Plot 2: Precision vs Recall (colored by FP count)
ax = axes[0, 1]
scatter = ax.scatter(config_metrics['overall_recall'], config_metrics['overall_precision'],
                     s=120, alpha=0.6, c=config_metrics['total_fp'], cmap='Reds_r')
ax.set_xlabel('Recall (Sensitivity)', fontsize=12)
ax.set_ylabel('Precision (1 - FPR)', fontsize=12)
ax.set_title('Precision-Recall Trade-off\n(color = false positives, darker = worse)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.set_xlim([0.0, 1.05])
ax.set_ylim([0.85, 1.05])
plt.colorbar(scatter, ax=ax, label='False Positives')
ax.annotate('baseline', 
            (baseline_row['overall_recall'], baseline_row['overall_precision']),
            fontsize=9, color='red')

# Plot 3: Dip vs Jump False Positives
ax = axes[1, 0]
ax.scatter(config_metrics['dip_fp'], config_metrics['jump_fp'], 
           s=120, alpha=0.6, c=config_metrics['grid_size'], cmap='plasma')
ax.set_xlabel('Dip False Positives', fontsize=12)
ax.set_ylabel('Jump False Positives', fontsize=12)
ax.set_title('Dip vs Jump False Positive Comparison', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.plot([0, ax.get_xlim()[1]], [0, ax.get_xlim()[1]], 'k--', alpha=0.3, label='Equal FP rate')
ax.legend()

# Plot 4: Speed vs False Positives (Pareto frontier)
ax = axes[1, 1]
# Normalize to baseline speed
baseline_time = baseline_row['elapsed_s']
config_metrics['speedup'] = baseline_time / config_metrics['elapsed_s']
scatter = ax.scatter(config_metrics['speedup'], config_metrics['total_fp'],
                     s=120, alpha=0.6, c=config_metrics['overall_f1'], cmap='RdYlGn')
ax.set_xlabel('Speedup vs Baseline', fontsize=12)
ax.set_ylabel('Total False Positives', fontsize=12)
ax.set_title('Speed vs False Positive Trade-off\n(color = F1 score, green = better)', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.axhline(baseline_row['total_fp'], color='red', linestyle='--', alpha=0.5, label='Baseline FP count')
ax.axvline(1.0, color='red', linestyle='--', alpha=0.5, label='Baseline speed')
ax.legend()
plt.colorbar(scatter, ax=ax, label='F1 Score')

# Plot 5: False Positive Rate by Grid Size Category
ax = axes[2, 0]
fp_by_size_plot = fp_by_size.reset_index()
x_pos = np.arange(len(fp_by_size_plot))
width = 0.35
ax.bar(x_pos - width/2, fp_by_size_plot['dip_fp'], width, label='Dip FP', alpha=0.8, color='#d62728')
ax.bar(x_pos + width/2, fp_by_size_plot['jump_fp'], width, label='Jump FP', alpha=0.8, color='#ff7f0e')
ax.set_xlabel('Grid Size Category', fontsize=12)
ax.set_ylabel('Average False Positives', fontsize=12)
ax.set_title('False Positive Rates by Grid Size', fontsize=14, fontweight='bold')
ax.set_xticks(x_pos)
ax.set_xticklabels(fp_by_size_plot['size_category'], rotation=15, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

# Plot 6: ROC-style curve (FPR vs TPR for configs)
ax = axes[2, 1]
# Calculate TPR (True Positive Rate = Recall)
config_metrics['tpr'] = config_metrics['overall_recall']
config_metrics['fpr'] = config_metrics['total_fp'] / len(simulated_lcs)  # FP rate normalized

scatter = ax.scatter(config_metrics['fpr'], config_metrics['tpr'],
                     s=120, alpha=0.6, c=config_metrics['grid_size'], cmap='viridis')
ax.set_xlabel('False Positive Rate', fontsize=12)
ax.set_ylabel('True Positive Rate (Recall)', fontsize=12)
ax.set_title('ROC-style: Detection Rate vs False Positive Rate', fontsize=14, fontweight='bold')
ax.grid(True, alpha=0.3)
ax.plot([0, 1], [0, 1], 'k--', alpha=0.3, label='Random classifier')
# Annotate best performers (high TPR, low FPR)
best_configs = config_metrics.nsmallest(5, 'fpr')
for idx, row in best_configs.iterrows():
    if row['tpr'] > 0.90:  # Only annotate if high recall
        ax.annotate(row['config'].replace('_', ' ')[:15], 
                   (row['fpr'], row['tpr']),
                   fontsize=7, alpha=0.7)
ax.legend()
plt.colorbar(scatter, ax=ax, label='Grid Size')

plt.tight_layout()
plt.show()

print("\n" + "="*80)
print("VISUALIZATION COMPLETE")
print("="*80)

In [ ]:
# Final recommendations based on false positive analysis
print("="*80)
print("RECOMMENDATIONS: OPTIMAL GRID CONFIGURATIONS")
print("="*80)

# Find configs that are Pareto-optimal on multiple fronts
# Criteria: low FP, high recall, high precision, fast speed

# Normalize metrics to [0, 1] for comparison
from sklearn.preprocessing import MinMaxScaler

metrics_for_scoring = config_metrics[['total_fp', 'overall_recall', 'overall_precision', 
                                       'elapsed_s', 'overall_f1']].copy()
scaler = MinMaxScaler()
metrics_normalized = pd.DataFrame(
    scaler.fit_transform(metrics_for_scoring),
    columns=metrics_for_scoring.columns,
    index=config_metrics.index
)

# Composite score: minimize FP, minimize time, maximize recall/precision/F1
# Weight FP heavily since that's the emphasis
config_metrics['composite_score'] = (
    -2.0 * metrics_normalized['total_fp'] +      # Weight FP 2x (lower is better)
    -0.5 * metrics_normalized['elapsed_s'] +     # Speed matters but less
    1.0 * metrics_normalized['overall_recall'] + # Maximize recall
    1.5 * metrics_normalized['overall_precision'] + # Weight precision heavily
    1.0 * metrics_normalized['overall_f1']       # Overall quality
)

top_configs = config_metrics.nlargest(10, 'composite_score')

print("\nTop 10 overall configurations (balanced scoring with emphasis on low FP):")
print(top_configs[['config', 'grid_size', 'total_fp', 'dip_fp', 'jump_fp',
                   'overall_precision', 'overall_recall', 'overall_f1', 
                   'elapsed_s', 'speedup', 'composite_score']].to_string())

# Best config for MINIMUM false positives (regardless of other factors)
best_fp = config_metrics.nsmallest(1, 'total_fp').iloc[0]
print(f"\n\n{'='*80}")
print("BEST FOR MINIMUM FALSE POSITIVES:")
print(f"{'='*80}")
print(f"Config: {best_fp['config']}")
print(f"Grid size: {int(best_fp['p_points'])}×{int(best_fp['mag_points'])} = {int(best_fp['grid_size'])} points")
print(f"False positives: {int(best_fp['total_fp'])} (dip: {int(best_fp['dip_fp'])}, jump: {int(best_fp['jump_fp'])})")
print(f"Precision: {best_fp['overall_precision']:.4f}")
print(f"Recall: {best_fp['overall_recall']:.4f}")
print(f"F1 Score: {best_fp['overall_f1']:.4f}")
print(f"Speed: {best_fp['elapsed_s']:.4f}s (speedup: {best_fp['speedup']:.2f}x)")

# Best config balancing speed and low FP
fast_accurate = config_metrics[
    (config_metrics['speedup'] > 2.0) &  # At least 2x faster
    (config_metrics['overall_recall'] > 0.90)  # High recall
].nsmallest(1, 'total_fp')

if len(fast_accurate) > 0:
    best_fast = fast_accurate.iloc[0]
    print(f"\n\n{'='*80}")
    print("BEST FAST CONFIG (>2x speedup, high recall, low FP):")
    print(f"{'='*80}")
    print(f"Config: {best_fast['config']}")
    print(f"Grid size: {int(best_fast['p_points'])}×{int(best_fast['mag_points'])} = {int(best_fast['grid_size'])} points")
    print(f"False positives: {int(best_fast['total_fp'])} (dip: {int(best_fast['dip_fp'])}, jump: {int(best_fast['jump_fp'])})")
    print(f"Precision: {best_fast['overall_precision']:.4f}")
    print(f"Recall: {best_fast['overall_recall']:.4f}")
    print(f"F1 Score: {best_fast['overall_f1']:.4f}")
    print(f"Speed: {best_fast['elapsed_s']:.4f}s (speedup: {best_fast['speedup']:.2f}x)")

# Baseline comparison
print(f"\n\n{'='*80}")
print("BASELINE (80×60) PERFORMANCE:")
print(f"{'='*80}")
print(f"False positives: {int(baseline_row['total_fp'])} (dip: {int(baseline_row['dip_fp'])}, jump: {int(baseline_row['jump_fp'])})")
print(f"Precision: {baseline_row['overall_precision']:.4f}")
print(f"Recall: {baseline_row['overall_recall']:.4f}")
print(f"F1 Score: {baseline_row['overall_f1']:.4f}")
print(f"Speed: {baseline_row['elapsed_s']:.4f}s (reference)")

# Summary statistics
print(f"\n\n{'='*80}")
print("SUMMARY STATISTICS ACROSS ALL CONFIGS:")
print(f"{'='*80}")
print(f"Total configs tested: {len(config_metrics)}")
print(f"Grid sizes: {int(config_metrics['grid_size'].min())} to {int(config_metrics['grid_size'].max())}")
print(f"\nFalse Positives:")
print(f"  Min: {int(config_metrics['total_fp'].min())}")
print(f"  Max: {int(config_metrics['total_fp'].max())}")
print(f"  Mean: {config_metrics['total_fp'].mean():.1f}")
print(f"  Median: {config_metrics['total_fp'].median():.1f}")
print(f"\nPrecision:")
print(f"  Min: {config_metrics['overall_precision'].min():.4f}")
print(f"  Max: {config_metrics['overall_precision'].max():.4f}")
print(f"  Mean: {config_metrics['overall_precision'].mean():.4f}")
print(f"\nRecall:")
print(f"  Min: {config_metrics['overall_recall'].min():.4f}")
print(f"  Max: {config_metrics['overall_recall'].max():.4f}")
print(f"  Mean: {config_metrics['overall_recall'].mean():.4f}")
print(f"\nSpeed (relative to baseline):")
print(f"  Fastest: {config_metrics['speedup'].max():.2f}x")
print(f"  Slowest: {config_metrics['speedup'].min():.2f}x")

In [ ]:
# Identify problematic light curves that consistently trigger false positives
print("="*80)
print("PROBLEMATIC LIGHT CURVES (HIGH FALSE POSITIVE TRIGGERS)")
print("="*80)

# Count how many configs trigger FP on each light curve
fp_trigger_counts = df_metrics[df_metrics['any_fp'] > 0].groupby('case').size().sort_values(ascending=False)

print(f"\nLight curves that trigger false positives in >50% of configs:")
threshold = len(config_metrics) * 0.5
problematic_lcs = fp_trigger_counts[fp_trigger_counts > threshold]

if len(problematic_lcs) > 0:
    print(f"Found {len(problematic_lcs)} problematic cases:\n")
    for lc_name, fp_count in problematic_lcs.items():
        pct = (fp_count / len(config_metrics)) * 100
        truth = ground_truth[lc_name]
        print(f"  {lc_name:50s} | Truth: {truth:15s} | FP in {fp_count:2d}/{len(config_metrics)} configs ({pct:.1f}%)")
    
    # Show details for most problematic case
    worst_case = problematic_lcs.index[0]
    print(f"\n\nDetails for most problematic case: {worst_case}")
    print(f"Ground truth: {ground_truth[worst_case]}")
    
    case_results = df_metrics[df_metrics['case'] == worst_case]
    fp_configs = case_results[case_results['any_fp'] > 0].sort_values('config')
    
    print(f"\nConfigs that falsely detected ({len(fp_configs)}/{len(config_metrics)}):")
    print(fp_configs[['config', 'dip_fp', 'jump_fp', 'dip_bf', 'jump_bf']].head(15))
    
else:
    print("\n✓ No consistently problematic light curves found!")
    print("  All false positives are isolated to specific configs.\n")

# Identify light curves that NEVER trigger false positives (most robust test cases)
never_fp = []
for lc_name in simulated_lcs.keys():
    case_fps = df_metrics[df_metrics['case'] == lc_name]['any_fp'].sum()
    if case_fps == 0:
        never_fp.append(lc_name)

print(f"\n{'='*80}")
print(f"ROBUST TEST CASES (never trigger false positives in any config):")
print(f"{'='*80}")
if len(never_fp) > 0:
    print(f"Found {len(never_fp)} cases that never trigger FPs:\n")
    for lc_name in never_fp[:20]:  # Show first 20
        truth = ground_truth[lc_name]
        print(f"  {lc_name:50s} | Truth: {truth}")
    if len(never_fp) > 20:
        print(f"  ... and {len(never_fp) - 20} more")
else:
    print("All light curves trigger at least one false positive in some configuration.")

# Analysis: Which event types are most/least prone to FP?
print(f"\n{'='*80}")
print("FALSE POSITIVE SUSCEPTIBILITY BY EVENT TYPE:")
print(f"{'='*80}")

fp_by_event_type = {}
for truth_type in set(ground_truth.values()):
    matching_cases = [k for k, v in ground_truth.items() if v == truth_type]
    total_possible_fps = len(matching_cases) * len(config_metrics)
    actual_fps = df_metrics[df_metrics['case'].isin(matching_cases)]['any_fp'].sum()
    fp_rate = (actual_fps / total_possible_fps) * 100 if total_possible_fps > 0 else 0
    
    fp_by_event_type[truth_type] = {
        'n_cases': len(matching_cases),
        'total_fps': actual_fps,
        'fp_rate': fp_rate
    }

for event_type, stats in sorted(fp_by_event_type.items(), key=lambda x: x[1]['fp_rate'], reverse=True):
    print(f"\n{event_type:20s}:")
    print(f"  # cases: {stats['n_cases']}")
    print(f"  Total FPs: {stats['total_fps']} / {stats['n_cases'] * len(config_metrics)} possible")
    print(f"  FP rate: {stats['fp_rate']:.2f}%")

## Next Steps

Based on the false positive analysis above, you can now:

1. **Choose your grid configuration** based on your priorities:
   - **Minimum false positives**: Use the config identified in "BEST FOR MINIMUM FALSE POSITIVES"
   - **Speed + accuracy balance**: Use the "BEST FAST CONFIG" recommendation
   - **Custom criteria**: Review the top 10 configs and choose based on your specific needs

2. **Update your pipeline** with the selected `p_points` and `mag_points` values

3. **Investigate problematic cases**: If certain light curves consistently trigger false positives, consider:
   - Adjusting significance thresholds
   - Examining those specific light curve patterns in detail
   - Adding additional filtering criteria

4. **Validate on real data**: Test the selected configuration on real ASAS-SN light curves to confirm performance

In [ ]:
# Define and run 65 extended grid configurations
# From absurdly coarse (2x2) to fine (120x100)

extended_grid_configs = []

# Absurdly coarse (2x2 to 10x10)
for n in [2, 3, 4, 5, 6, 7, 8, 9, 10]:
    extended_grid_configs.append({"label": f"absurd_{n}x{n}", "p_points": n, "mag_points": n})

# Ultra coarse (12x12 to 20x20)
for n in [12, 15, 18, 20]:
    extended_grid_configs.append({"label": f"ultra_coarse_{n}x{n}", "p_points": n, "mag_points": n})

# Very coarse (22x22 to 35x35)
for n in [22, 25, 28, 30, 35]:
    extended_grid_configs.append({"label": f"very_coarse_{n}x{n}", "p_points": n, "mag_points": n})

# Coarse (asymmetric versions)
for p, m in [(20, 15), (25, 20), (30, 20), (30, 25), (35, 25), (40, 25), (40, 30), (45, 30), (50, 35)]:
    extended_grid_configs.append({"label": f"coarse_{p}x{m}", "p_points": p, "mag_points": m})

# Medium (50x40 to 70x60)
for p, m in [(50, 40), (55, 45), (60, 45), (60, 50), (65, 50), (70, 50), (70, 60)]:
    extended_grid_configs.append({"label": f"medium_{p}x{m}", "p_points": p, "mag_points": m})

# Baseline and near-baseline
extended_grid_configs.append({"label": "baseline_80x60", "p_points": 80, "mag_points": 60})
for p, m in [(75, 55), (85, 65), (90, 65), (90, 70)]:
    extended_grid_configs.append({"label": f"near_base_{p}x{m}", "p_points": p, "mag_points": m})

# Fine (95x70 to 120x100)
for p, m in [(95, 70), (100, 75), (100, 80), (105, 80), (110, 85), (110, 90), (115, 90), (120, 90), (120, 100)]:
    extended_grid_configs.append({"label": f"fine_{p}x{m}", "p_points": p, "mag_points": m})

print(f"Total configurations: {len(extended_grid_configs)}")
print(f"Grid sizes range from {min(c['p_points']*c['mag_points'] for c in extended_grid_configs)} to {max(c['p_points']*c['mag_points'] for c in extended_grid_configs)}")
print(f"\nSample configs: {extended_grid_configs[:5]}")

# Run all configurations on real light curves
N_LC_EXTENDED = 3
subset_paths_extended = lc_paths[:N_LC_EXTENDED]

extended_rows = []
print(f"\nRunning {len(extended_grid_configs)} configs on {N_LC_EXTENDED} light curves...")
for i, path in enumerate(subset_paths_extended):
    print(f"  Light curve {i+1}/{N_LC_EXTENDED}: {path.name}")
    for j, cfg in enumerate(extended_grid_configs):
        if (j + 1) % 10 == 0:
            print(f"    Config {j+1}/{len(extended_grid_configs)}")
        try:
            extended_rows.append(run_grid_setting(path, cfg["p_points"], cfg["mag_points"], cfg["label"]))
        except Exception as e:
            print(f"    ERROR on {cfg['label']}: {e}")

df_extended = pd.DataFrame(extended_rows)
print(f"\nCompleted: {len(df_extended)} results")
df_extended.head(10)

In [ ]:
# Analyze extended grid results relative to baseline
baseline_label = "baseline_80x60"

# Compute average metrics across all light curves for each config
summary = df_extended.groupby("config").agg({
    "elapsed_s": "mean",
    "p_points": "first",
    "mag_points": "first",
    "dip_sig": "sum",  # Count how many were significant
    "jump_sig": "sum",
    "dip_bf": "mean",
    "jump_bf": "mean",
}).reset_index()

# Get baseline values
baseline = summary[summary["config"] == baseline_label].iloc[0]

# Compute relative metrics
summary["speedup"] = baseline["elapsed_s"] / summary["elapsed_s"]
summary["dip_bf_ratio"] = summary["dip_bf"] / baseline["dip_bf"]
summary["jump_bf_ratio"] = summary["jump_bf"] / baseline["jump_bf"]
summary["grid_size"] = summary["p_points"] * summary["mag_points"]

# Sort by speed (fastest first)
summary_sorted = summary.sort_values("elapsed_s")

print(f"\\nBaseline (80x60): {baseline['elapsed_s']:.3f}s average")
print(f"\\nTop 10 fastest configurations:")
print(summary_sorted[["config", "p_points", "mag_points", "grid_size", "elapsed_s", "speedup", 
                       "dip_sig", "jump_sig", "dip_bf_ratio", "jump_bf_ratio"]].head(10))

print(f"\\n\\nFull ranking by speed:")
summary_sorted[["config", "elapsed_s", "speedup", "dip_sig", "jump_sig", "dip_bf_ratio", "jump_bf_ratio"]]

In [ ]:
# Visualize speed vs accuracy tradeoff
import matplotlib.pyplot as plt

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Plot 1: Grid size vs elapsed time
axes[0, 0].scatter(summary["grid_size"], summary["elapsed_s"], s=100, alpha=0.6, c=summary["speedup"], cmap="viridis")
axes[0, 0].set_xlabel("Grid Size (p_points × mag_points)")
axes[0, 0].set_ylabel("Elapsed Time (s)")
axes[0, 0].set_title("Grid Size vs Speed")
axes[0, 0].grid(True, alpha=0.3)
# Annotate a few key points
for idx, row in summary.iterrows():
    if row["config"] in ["ultra_coarse_10x10", "coarse_30x30", "coarse_40x30", "baseline_80x60"]:
        axes[0, 0].annotate(row["config"].replace("_", " "), 
                           (row["grid_size"], row["elapsed_s"]),
                           fontsize=8, alpha=0.7)

# Plot 2: Speedup vs Bayes factor accuracy (dip)
axes[0, 1].scatter(summary["speedup"], summary["dip_bf_ratio"], s=100, alpha=0.6, c=summary["grid_size"], cmap="plasma")
axes[0, 1].axhline(1.0, color='red', linestyle='--', alpha=0.5, label="Perfect accuracy")
axes[0, 1].axhline(0.99, color='orange', linestyle='--', alpha=0.5, label="99% accuracy")
axes[0, 1].set_xlabel("Speedup vs Baseline")
axes[0, 1].set_ylabel("Dip Bayes Factor Ratio")
axes[0, 1].set_title("Speed vs Dip BF Accuracy")
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Plot 3: Speedup vs Bayes factor accuracy (jump)
axes[1, 0].scatter(summary["speedup"], summary["jump_bf_ratio"], s=100, alpha=0.6, c=summary["grid_size"], cmap="plasma")
axes[1, 0].axhline(1.0, color='red', linestyle='--', alpha=0.5, label="Perfect accuracy")
axes[1, 0].axhline(0.90, color='orange', linestyle='--', alpha=0.5, label="90% accuracy")
axes[1, 0].set_xlabel("Speedup vs Baseline")
axes[1, 0].set_ylabel("Jump Bayes Factor Ratio")
axes[1, 0].set_title("Speed vs Jump BF Accuracy")
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Plot 4: Pareto frontier (speedup vs combined BF error)
summary["bf_error"] = np.abs(1 - summary["dip_bf_ratio"]) + np.abs(1 - summary["jump_bf_ratio"])
axes[1, 1].scatter(summary["speedup"], summary["bf_error"], s=100, alpha=0.6, c=summary["grid_size"], cmap="coolwarm")
axes[1, 1].set_xlabel("Speedup vs Baseline")
axes[1, 1].set_ylabel("Combined BF Error (lower is better)")
axes[1, 1].set_title("Speed vs Combined Accuracy")
axes[1, 1].set_yscale("log")
axes[1, 1].grid(True, alpha=0.3)
# Annotate sweet spot
for idx, row in summary.iterrows():
    if row["config"] in ["coarse_30x30", "coarse_40x30", "baseline_80x60"]:
        axes[1, 1].annotate(row["config"].replace("_", " "), 
                           (row["speedup"], row["bf_error"]),
                           fontsize=8, alpha=0.7)

plt.tight_layout()
plt.show()

# Identify the "sweet spot" configurations
print("\\n" + "="*80)
print("SWEET SPOT ANALYSIS")
print("="*80)
print("\\nConfigurations with >3x speedup and <5% BF error:")
sweet_spot = summary[(summary["speedup"] > 3) & (summary["bf_error"] < 0.05)]
if len(sweet_spot) > 0:
    print(sweet_spot[["config", "p_points", "mag_points", "speedup", "elapsed_s", "bf_error"]].sort_values("speedup", ascending=False))
else:
    print("No configurations meet these criteria. Relaxing to >2x speedup and <10% error:")
    sweet_spot = summary[(summary["speedup"] > 2) & (summary["bf_error"] < 0.10)]
    print(sweet_spot[["config", "p_points", "mag_points", "speedup", "elapsed_s", "bf_error"]].sort_values("speedup", ascending=False))

## Simulated dips and jumps
Create a small suite of synthetic light curves (reusing a real SkyPatrol cadence) with injected dips, jumps, and a mixed case so grid sensitivity can be tested on known events.

In [ ]:
# Build simulated light curves by injecting analytic events into a real cadence
from malca import events

rng = np.random.default_rng(42)

base_sim_path = lc_paths[0]
df_sim_base_raw = read_skypatrol_csv(base_sim_path)
mask = (
    df_sim_base_raw["JD"].pipe(np.isfinite)
    & df_sim_base_raw["mag"].pipe(np.isfinite)
    & df_sim_base_raw["error"].pipe(np.isfinite)
    & (df_sim_base_raw["error"] > 0)
    & (df_sim_base_raw["error"] < 10)
)
df_sim_base = df_sim_base_raw[mask].copy()
df_sim_base = events.clean_lc(df_sim_base)

jd_med = float(df_sim_base["JD"].median())


def inject_event(df_in, kind="dip", shape="gaussian", amp=0.25, width=25.0, t0_offset=0.0):
    # Return a copy with an injected event (positive amp = dip, negative amp = jump).
    df = df_in.copy()
    t0 = jd_med + float(t0_offset)
    amp_signed = float(amp) if kind == "dip" else -float(amp)
    t_arr = df["JD"].to_numpy(float)
    if shape == "gaussian":
        delta = events.gaussian(t_arr, amp_signed, t0, float(width), 0.0)
    elif shape == "paczynski":
        delta = events.paczynski(t_arr, amp_signed, t0, float(width), 0.0)
    else:
        raise ValueError("shape must be gaussian or paczynski")
    df["mag"] = df["mag"].to_numpy(float) + delta
    return df


def inject_mixed(df_in):
    df = inject_event(df_in, kind="dip", shape="gaussian", amp=0.18, width=20.0, t0_offset=-60.0)
    df = inject_event(df, kind="jump", shape="paczynski", amp=0.14, width=30.0, t0_offset=40.0)
    return df


def add_noise_and_gaps(df_in, jitter_mag=0.02, gap_frac=0.15, spike_amp=0.25, spike_width=0.08, spike_count=3):
    # Add photometric jitter, drop random points, and sprinkle outliers to make messy cases.
    df = df_in.copy()
    n = len(df)
    df["mag"] = df["mag"].to_numpy(float) + rng.normal(0.0, jitter_mag, n)
    df["error"] = (df["error"].to_numpy(float) * (1 + rng.normal(0.0, 0.05, n))).clip(min=1e-3)
    keep = rng.random(n) > gap_frac
    df = df.loc[keep].copy().reset_index(drop=True)
    if len(df) == 0:
        return df_in.copy()
    for _ in range(int(spike_count)):
        idx = int(rng.integers(0, len(df)))
        df.loc[idx, "mag"] += rng.normal(spike_amp, spike_width)
    df = df.sort_values("JD").reset_index(drop=True)
    return df

simulated_lcs = {
    "dip_gaussian": inject_event(df_sim_base, kind="dip", shape="gaussian", amp=0.22, width=25.0, t0_offset=-30.0),
    "dip_paczynski": inject_event(df_sim_base, kind="dip", shape="paczynski", amp=0.28, width=18.0, t0_offset=20.0),
    "dip_shallow_fast": inject_event(df_sim_base, kind="dip", shape="gaussian", amp=0.12, width=8.0, t0_offset=-10.0),
    "dip_double": inject_event(inject_event(df_sim_base, kind="dip", shape="gaussian", amp=0.18, width=15.0, t0_offset=-50.0), kind="dip", shape="paczynski", amp=0.15, width=12.0, t0_offset=25.0),
    "jump_gaussian": inject_event(df_sim_base, kind="jump", shape="gaussian", amp=0.20, width=22.0, t0_offset=-15.0),
    "jump_paczynski": inject_event(df_sim_base, kind="jump", shape="paczynski", amp=0.24, width=16.0, t0_offset=45.0),
    "microlens_weak": inject_event(df_sim_base, kind="jump", shape="paczynski", amp=0.10, width=10.0, t0_offset=5.0),
    "microlens_strong": inject_event(df_sim_base, kind="jump", shape="paczynski", amp=0.30, width=26.0, t0_offset=70.0),
    "mixed": inject_mixed(df_sim_base),
    "messy_dip": add_noise_and_gaps(inject_event(df_sim_base, kind="dip", shape="gaussian", amp=0.20, width=18.0, t0_offset=-5.0), jitter_mag=0.03, gap_frac=0.2, spike_count=3),
    "messy_jump": add_noise_and_gaps(inject_event(df_sim_base, kind="jump", shape="paczynski", amp=0.18, width=14.0, t0_offset=35.0), jitter_mag=0.03, gap_frac=0.2, spike_count=3),
    "messy_mixed": add_noise_and_gaps(inject_mixed(df_sim_base), jitter_mag=0.04, gap_frac=0.25, spike_count=4),
}

print(f"Simulated light curves: {list(simulated_lcs.keys())}")
display(df_sim_base.head())

In [ ]:
from malca.baseline import per_camera_gp_baseline


def prepare_df_for_sim(df):
    df_clean = events.clean_lc(df)
    df_base = per_camera_gp_baseline(df_clean, **BASELINE_KWARGS)
    baseline_mags = df_base["baseline"].to_numpy(float) if "baseline" in df_base.columns else df_base["mag"].to_numpy(float)
    baseline_mag = float(np.nanmedian(baseline_mags))
    mags_for_grid = df_base["mag"].to_numpy(float) if "mag" in df_base.columns else df_clean["mag"].to_numpy(float)

    def baseline_precomputed(df_in, **kwargs):
        return df_base

    return df_clean, baseline_precomputed, baseline_mag, mags_for_grid


def run_grid_setting_sim(name, df, p_points, mag_points, label):
    df_clean, baseline_fn, baseline_mag, mags_for_grid = prepare_df_for_sim(df)
    mag_grid_dip = events.default_mag_grid(baseline_mag, mags_for_grid, "dip", n=mag_points)
    mag_grid_jump = events.default_mag_grid(baseline_mag, mags_for_grid, "jump", n=mag_points)

    start = time.perf_counter()
    res = events.run_bayesian_significance(
        df_clean,
        baseline_func=baseline_fn,
        baseline_kwargs={},
        p_points=p_points,
        mag_grid_dip=mag_grid_dip,
        mag_grid_jump=mag_grid_jump,
        trigger_mode="posterior_prob",
        logbf_threshold_dip=5.0,
        logbf_threshold_jump=5.0,
        significance_threshold=99.99997,
        run_min_points=3,
        run_allow_gap_points=1,
        run_max_gap_days=None,
        run_min_duration_days=None,
        use_sigma_eff=True,
        require_sigma_eff=True,
        compute_event_prob=True,
    )
    elapsed = time.perf_counter() - start

    return {
        "case": name,
        "config": label,
        "p_points": p_points,
        "mag_points": mag_points,
        "elapsed_s": elapsed,
        "dip_sig": res["dip"]["significant"],
        "jump_sig": res["jump"]["significant"],
        "dip_bf": res["dip"]["bayes_factor"],
        "jump_bf": res["jump"]["bayes_factor"],
        "dip_best_p": res["dip"]["best_p"],
        "jump_best_p": res["jump"]["best_p"],
    }

In [ ]:
# Sweep grid sizes on the simulated cases
sim_grid_settings = [
    {"label": "sim_10x10", "p_points": 10, "mag_points": 10},
    {"label": "sim_25x25", "p_points": 25, "mag_points": 25},
    {"label": "sim_50x50", "p_points": 50, "mag_points": 50},
    {"label": "sim_80x60", "p_points": 80, "mag_points": 60},
]

sim_rows = []
for name, df_sim in simulated_lcs.items():
    for cfg in sim_grid_settings:
        sim_rows.append(run_grid_setting_sim(name, df_sim, cfg["p_points"], cfg["mag_points"], cfg["label"]))

sim_results = pd.DataFrame(sim_rows)
sim_results